In [1]:
# Operações matemáticas.
import numpy as np

# Plotagem gráfica.
import matplotlib.pyplot as plt

# Pré-processamento.
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Rede Neural e componentes.
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomZoom
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall

In [2]:
# Semente de reprodução.
tf.random.set_seed(4)

#### Carregando o caminho das imagens

In [3]:
# Carregando caminhos.
train_path = Path("../fruits-360/Training")
test_path = Path("../fruits-360/Test")

# Recuperando o nome de todos as pastas e seus arquivos.
train_images = list(train_path.glob("*/*"))
train_images = list(map(lambda x: str(x), train_images))

train_images[:5]

['..\\fruits-360\\Training\\Apple Braeburn\\0_100.jpg',
 '..\\fruits-360\\Training\\Apple Braeburn\\100_100.jpg',
 '..\\fruits-360\\Training\\Apple Braeburn\\101_100.jpg',
 '..\\fruits-360\\Training\\Apple Braeburn\\102_100.jpg',
 '..\\fruits-360\\Training\\Apple Braeburn\\103_100.jpg']

### Pré-processamento

In [4]:
# Extraindo labels.
extract_file = lambda x: x.split("\\")[-2]
train_labels = list(map(lambda x: extract_file(x), train_images))

train_labels[:5]

['Apple Braeburn',
 'Apple Braeburn',
 'Apple Braeburn',
 'Apple Braeburn',
 'Apple Braeburn']

In [5]:
# Codificação das labels.
encoder = LabelEncoder()
train_labels = encoder.fit_transform(train_labels)

# One-hot encoding.
train_labels = tf.keras.utils.to_categorical(train_labels)

train_labels[:3]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [6]:
# Separando em conjuntos de treino e validação
X_train, X_valid, y_train, y_valid = train_test_split(train_images, train_labels)

In [7]:
# Criando função de redimensionamento.
img_size = 224
resize = tf.keras.Sequential([tf.keras.layers.experimental.preprocessing.Resizing(img_size, img_size)])

# Dataset Augmentation
data_augmentation = tf.keras.Sequential([RandomFlip("horizontal"),  # Inversão;
                                         RandomRotation(0.2),       # Rotação;
                                         RandomZoom((-0.3, -0.2))]) # Zoom.

In [8]:
# Tamanho dos lotes de imagens a serem treinados.
batch_size = 32

In [9]:
# Função para carregar imagem.
def load_transform(image):
    
    # Lê a imagem.
    image = tf.io.read_file(image)
    
    # Decodifica de JPEG para um tensor de com 3 canais de cores.
    image = tf.io.decode_jpeg(image, channels=3)
    
    return image

In [10]:
# Função para preparar os dados.
def prepare_dataset(paths, labels, train=True):
    
    # Conversão para tensores.
    image_paths = tf.convert_to_tensor(paths)
    labels = tf.convert_to_tensor(labels)
    
    # Criando datasets a partir dos tensores.
    image_dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    label_dataset = tf.data.Dataset.from_tensor_slices(labels)
    
    # Pareando a imagem a seu respectivo rótulo.
    dataset = tf.data.Dataset.zip((image_dataset, label_dataset))
    
    # Carregando e redimensionando imagem.
    dataset = dataset.map(lambda image, label: (load_transform(image), label))
    dataset = dataset.map(lambda image, label: (resize(image), label), 
                          num_parallel_calls=tf.data.experimental.AUTOTUNE)
    
    # Embaralhando os dados e formando os lotes.
    dataset = dataset.shuffle(1000)
    dataset = dataset.batch(batch_size)
    
    # Se os dados forem de treino, então fazemos o Data Augmentation.
    if train:
        dataset = dataset.map(lambda image, label: (data_augmentation(image), label), 
                              num_parallel_calls=tf.data.experimental.AUTOTUNE)
        
    return dataset

In [11]:
# Preparando os conjuntos.
train_dataset = prepare_dataset(X_train, y_train)
valid_dataset = prepare_dataset(X_valid, y_valid, train=False)

In [12]:
image, label = next(iter(train_dataset))

image.shape, label.shape

(TensorShape([32, 224, 224, 3]), TensorShape([32, 131]))

##### Cada lote tem 32 imagens, cada imagem tem 224 x 224 pixels e 3 canais de cores, além de 131 rótulos disponíveis.

### Modelo Convolucional

#### Construindo o modelo

In [13]:
model = tf.keras.Sequential(
    [
        # Arquitetura da rede neural, sem a última camada.
        EfficientNetB3(input_shape=(224, 224, 3), include_top=False),
        
        # Redução para 2 dimensões.
        tf.keras.layers.GlobalAveragePooling2D(),
        
        # Adicionando a última camada, com as 131 classes disponíveis.
        tf.keras.layers.Dense(131, activation="softmax")
    ]
)

In [14]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb3 (Functional)  (None, 7, 7, 1536)       10783535  
                                                                 
 global_average_pooling2d (G  (None, 1536)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 131)               201347    
                                                                 
Total params: 10,984,882
Trainable params: 10,897,579
Non-trainable params: 87,303
_________________________________________________________________


In [15]:
# Valores dos hiperparâmetros.
lr, beta1, beta2, ep = 1e-3, .9, .999, 1e-7

# Compilação do modelo.
model.compile(
    # Algoritmo de backpropagation.
    optimizer = Adam(learning_rate=lr, beta_1=beta1, beta_2=beta2, epsilon=ep),
    
    # Função de custo.
    loss = "categorical_crossentropy",
    
    # Métricas de avaliação.
    metrics = ["accuracy", Precision(name="precision"), Recall(name="recall")]
)

In [16]:
# Criando callbacks.

# Checkpoint para salvar os pesos do melhor modelo.
checkpoint = tf.keras.callbacks.ModelCheckpoint("../models/best_model.h5",
                                                verbose=1,
                                                save_best=True,
                                                save_weights_only=True)

# EarlyStopping com espera de 4 iterações.
early_stop = tf.keras.callbacks.EarlyStopping(patience=4)

In [ ]:
# Treinando modelo...
model.fit(train_dataset,
          steps_per_epoch = len(X_train)//batch_size,
          epochs = 6,
          validation_data = valid_dataset,
          validation_steps = len(y_train)//batch_size,
          callbacks = [checkpoint, early_stop])

In [17]:
# Carregando os pesos do modelo treinado.
model.load_weights("../models/best_model.h5")

#### Avaliando o modelo

In [18]:
# Aplicando o mesmo pré-processamento aos dados de teste.
test_images = list(test_path.glob("*/*"))
test_images = list(map(lambda x: str(x), test_images))
test_labels = list(map(lambda x: extract_file(x), test_images))
test_labels = encoder.fit_transform(test_labels)
test_labels = tf.keras.utils.to_categorical(test_labels)
test_image_paths = tf.convert_to_tensor(test_images)
test_image_labels = tf.convert_to_tensor(test_labels)

In [19]:
# Função para decodificar e redimensionar imagens.
def decode_images(image, label):
    
    image = tf.io.read_file(image)
    image = tf.io.decode_jpeg(image, channels = 3)
    image = tf.image.resize(image, [224, 224], method = "bilinear")
    
    return image, label

In [20]:
# Criando dataset de teste com as mesmas configurações que o dataset de treino.
test_dataset = (tf.data.Dataset
                .from_tensor_slices((test_images, test_labels))
                .map(decode_images)
                .batch(batch_size))

In [ ]:
loss, acc, prec, rec = model.evaluate(test_dataset)

#### Acurácia: 0.7616
#### Precisão: 0.7857
#### Revocação: 0.7499